# ⚙️ Set Up

In [1]:
# Import Python built-in libraries
import os
import copy
import pickle
import random
import time

In [2]:
# Import pip libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm, trange

# Import torch packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data

In [3]:
# Import PyG packages
import torch_geometric as pyg
import torch_geometric.data as pyg_data
from torch_geometric.typing import Adj, OptTensor


In [4]:
from sklearn.preprocessing import normalize

# ⚗️ Data Preprocessing

# 📦 Data Pipeline

For data ingestion, we use PyTorch's `dataloader` and PyG's `Data` class. To learn more about the `Data` class, check out the documentation [here](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#module-torch_geometric.data).

In [5]:
class GraphDataset(pyg_data.InMemoryDataset):
    def __init__(self, root, file_name, transform=None, pre_transform=None):
        self.file_name = file_name
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return [f'{self.file_name}.txt']

    @property
    def processed_file_names(self):
        return [f'{self.file_name}.pt']

    def download(self):
        pass

    def process(self):
        data_list = []

        counter = 0

        for session in sessions:
            codes, uniques = pd.factorize(session)
            senders, receivers = codes[:-1], codes[1:]

            # Build Data instance
            edge_index = torch.tensor([senders, receivers], dtype=torch.long)
            #x = torch.tensor(uniques, dtype=torch.long)
            x_new = torch.zeros((len(uniques), 100))

            item_counter = 0
            for item in uniques:
                x_new[item_counter] = torch.tensor(embeddings[item])
                item_counter += 1

            #y = torch.tensor([y], dtype=torch.long)

            data_list.append(pyg_data.Data(x=x_new, edge_index=edge_index))

            counter += 1

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# 🔮 Model Design

Our gated session graph layer has two main parts: (1) message propagation to create an adjacency matrix (`self.propagate`) and (2) the GRU cell (`self.gru`). We will put these inside the `forward()` function.

We only use one layer for our `GatedSessionGraphConv` implementation for simplicity. Also, our sessions have average length < 5, so we do not need a large receptive field.

In [6]:
class GatedSessionGraphConv(pyg.nn.conv.MessagePassing):
    def __init__(self, out_channels, aggr: str = 'add', **kwargs):
        super().__init__(aggr=aggr, **kwargs)

        self.out_channels = out_channels

        self.gru = torch.nn.GRUCell(out_channels, out_channels, bias=False)

    def forward(self, x, edge_index):
        m = self.propagate(edge_index, x=x, size=None)
        x = self.gru(m, x)
        return x

    def message(self, x_j):
        return x_j

    def message_and_aggregate(self, adj_t, x):
        return matmul(adj_t, x, reduce=self.aggr)

In [7]:
class SRGNN(nn.Module):
    def __init__(self, hidden_size, n_items):
        super(SRGNN, self).__init__()
        self.hidden_size = hidden_size
        self.n_items = n_items

        self.gated = GatedSessionGraphConv(self.hidden_size)

        self.q = nn.Linear(self.hidden_size, 1)
        self.W_1 = nn.Linear(self.hidden_size, self.hidden_size, bias=False)
        self.W_2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.W_3 = nn.Linear(2 * self.hidden_size, self.hidden_size, bias=False)

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)

    def forward(self, data):
        x, edge_index, batch_map = data.x, data.edge_index, data.batch

        # (0)
        #embedding = self.embedding(x).squeeze()

        # (1)-(5)
        v_i = self.gated(x, edge_index)

        # Divide nodes by session
        # For the detailed explanation of what is happening below, please refer
        # to the Medium blog post.
        sections = list(torch.bincount(batch_map).cpu())
        v_i_split = torch.split(v_i, sections)

        v_n, v_n_repeat = [], []
        for session in v_i_split:
            v_n.append(session[-1])
            v_n_repeat.append(
                session[-1].view(1, -1).repeat(session.shape[0], 1))
        v_n, v_n_repeat = torch.stack(v_n), torch.cat(v_n_repeat, dim=0)

        q1 = self.W_1(v_n_repeat)
        q2 = self.W_2(v_i)

        # (6)
        alpha = self.q(F.sigmoid(q1 + q2))
        s_g_split = torch.split(alpha * v_i, sections)

        s_g = []
        for session in s_g_split:
            s_g_session = torch.sum(session, dim=0)
            s_g.append(s_g_session)
        s_g = torch.stack(s_g)

        # (7)
        s_l = v_n
        s_h = self.W_3(torch.cat([s_l, s_g], dim=-1))
        #print("SH: ")
        #print(s_h.shape)
        #print(s_h)


        return s_h

# 🚂 Model Training

We can now start model training. The training pipeline code below was originally taken from the 2021 Fall CS224W Colab assignments and then modified to fit the model.

In [8]:
# Define the hyperparameters.
# Code taken from 2021 Fall CS224W Colab assignments.
args = {
    'batch_size': 100,
    'hidden_dim': 100,
    'epochs': 1,
    'l2_penalty': 0.00001,
    'weight_decay': 0.1,
    'step': 30,
    'lr': 0.001,
    'num_items': 466868}

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

args = objectview(args)

In [9]:
def test(loader, test_model,num_of_sessions, save_model_preds=False):
    test_model.eval()

    preds_all = torch.zeros(num_of_sessions, 100)
    i = 0

    for _, data in enumerate(tqdm(loader)):
        data
        with torch.no_grad():
            # max(dim=1) returns values, indices tuple; only need indices
            preds = test_model(data)
            preds_all[i*args.batch_size:(i+1)*args.batch_size,:] = preds
            i += 1

    if save_model_preds:
        np_preds = preds_all.cpu().detach().numpy()
        return np_preds


In [10]:
#CHANGE MODEL NAMES ACCORDINGLY !!!!
LOCALES = ["ES", "FR", "IT"]

with open("../SR-GNN/raw/united_composed_embedding.pkl", 'rb') as f:
        united_composed_embedding = pickle.load(f)

for i in range(3):
    locale = LOCALES[i]

    with open("../embeddings/{}-composed_embedding.pkl".format(locale), 'rb') as f:
        embeddings = pickle.load(f)

    for i in embeddings.keys():
        embeddings[i] = united_composed_embedding[i]

    with open('../data/preprocessed-data/{}-sessions-test.pkl'.format(locale), 'rb') as f:
        sessions = pickle.load(f)

    test_dataset = GraphDataset('./', locale)
    test_loader = pyg_data.DataLoader(test_dataset,
                                  batch_size=args.batch_size,
                                  shuffle=False,
                                  drop_last=True)

    model = SRGNN(args.hidden_dim, args.num_items)
    model.load_state_dict(torch.load("../model/united-model"))

    nextItems = test(test_loader, model,len(sessions), True)

    key_list = list(embeddings.keys())
    values_list = list(embeddings.values())
    stacked_embeddings = np.stack( values_list, axis=0)

    stacked_embeddings = normalize(stacked_embeddings, axis=1, norm='l2')
    nextItems = normalize(nextItems, axis=1, norm='l2')

    sim = np.dot(nextItems, stacked_embeddings.T)
    sortedSimilarity = np.argsort(sim, axis=1)
    sortedSimilarity = sortedSimilarity[:, -100:]
    recommendation_lists = []

    for x in range(sortedSimilarity.shape[0]):
        rec_sub_list = []
        for y in range(sortedSimilarity.shape[1]):
            rec_sub_list.append(key_list[sortedSimilarity[x, y]])
        rec_sub_list.reverse()
        recommendation_lists.append(rec_sub_list)

    with open('./test-result/{}-recs.pkl'.format(locale), 'wb') as f:
        pickle.dump(recommendation_lists, f)




C:\Users\dogat\anaconda3\envs\bitirmesolution\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
100%|██████████| 81/81 [00:03<00:00, 25.04it/s]
Processing...
C:\Users\dogat\AppData\Local\Temp\ipykernel_21652\277422053.py:28: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  edge_index = torch.tensor([senders, receivers], dtype=torch.long)
Done!
C:\Users\dogat\anaconda3\envs\bitirmesolution\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
100%|██████████| 125/125 [00:03<00:00, 41.29it/s]
Processing...
Done!
C:\Users\dogat\anaconda3\envs\bitirmesoluti